# Python sockets 101 - Client side
- ## Sébastien Tixeuil
- ## LINCS Python Workshop 14 December 2020

# Source
- <https://www.apress.com/gp/book/9781430258544>

![](https://media.springernature.com/w153/springer-static/cover/book/9781430258551.jpg)

# UDP Client (Python 2X)

In [ ]:
from socket import *
serverName = '127.0.0.1'
serverPort = 1235
clientSocket = socket(AF_INET,SOCK_DGRAM)
message = input('lowercase sentence:')
clientSocket.sendto(message,(serverName,serverPort))
modifiedMessage, serverAddress = clientSocket.recvfrom(2048)
print(modifiedMessage)
clientSocket.close()

# UDP Client (Python 3X)

In [ ]:
from socket import *
serverName = '127.0.0.1'
serverPort = 1235
clientSocket = socket(AF_INET,SOCK_DGRAM)
message = input('lowercase sentence:')
clientSocket.sendto(message.encode('utf-8'),(serverName,serverPort))
modifiedMessage, serverAddress = clientSocket.recvfrom(2048)
print(modifiedMessage.decode('utf-8'))
clientSocket.close()

# UDP Client (TTL Estimator)

In [ ]:
from socket import *
from time import *

serverName = 'localhost'
serverPort = 1234

clientSocket = socket(AF_INET,SOCK_DGRAM)

nb_samples = 1000
avg_TTL = 0
sum_TTL = 0

for i in range(nb_samples):
	start = time_ns()
	clientSocket.sendto(b"PING",(serverName,serverPort))
	clientSocket.recvfrom(2048)
	stop = time_ns()
	sum_TTL += stop - start

avg_TTL = sum_TTL / nb_samples

print(f"Average TTL from {nb_samples} samples is {avg_TTL}ns") 

clientSocket.close()

# TCP Client

In [ ]:
from socket import *
serverName = '127.0.0.1'
serverPort = 1235
clientSocket = socket(AF_INET,SOCK_STREAM)
clientSocket.connect((serverName,serverPort))
message = input('lowercase sentence:')
clientSocket.send(message.encode('utf-8'))
modifiedMessage = clientSocket.recv(2048)
print(modifiedMessage.decode('utf-8'))
clientSocket.close()

# UDP Client handling packet drops (Infinite Timer)

In [ ]:
from socket import *

serverName = '127.0.0.1'
serverPort = 1236
clientSocket = socket(AF_INET,SOCK_DGRAM)
message = input('lowercase sentence:')

delay = 0.1 # sec
while True:
    clientSocket.sendto(message.encode('utf-8'),(serverName,serverPort))
    clientSocket.settimeout(delay)
    try:
        modifiedMessage, serverAddress = clientSocket.recvfrom(2048)
    except timeout:
        delay *= 2
        print('delay doubled..')
    else:
        break
print(modifiedMessage.decode('utf-8'))
clientSocket.close()

# UDP Client handling packet drops (2 sec upper bound)

In [ ]:
from socket import *

serverName = '127.0.0.1'
serverPort = 1236
clientSocket = socket(AF_INET,SOCK_DGRAM)
message = input('lowercase sentence:')

delay = 0.1 # sec
while True:
    clientSocket.sendto(message.encode('utf-8'),(serverName,serverPort))
    clientSocket.settimeout(delay)
    try:
        modifiedMessage, serverAddress = clientSocket.recvfrom(2048)
    except timeout:
        delay *= 2
        print('delay doubled..')
        if delay > 2.0:
            raise RuntimeError('server seems down')
    else:
        break
print(modifiedMessage.decode('utf-8'))
clientSocket.close()

# UDP Broadcast Client

In [ ]:
from socket import *

broadcastAddr = '192.168.1.255'
serverPort = 1234
clientSocket = socket(AF_INET,SOCK_DGRAM)
clientSocket.setsockopt(SOL_SOCKET,SO_BROADCAST,1)
message = input('lowercase sentence:').encode('utf-8')
clientSocket.sendto(message,(serverName,serverPort))
modifiedMessage, serverAddress = clientSocket.recvfrom(2048)
print(modifiedMessage.decode('utf-8'))
clientSocket.close()

# TCP Send issues

- If TCP/IP stack has enough room, `send` returns immediately, and the **complete** message will be handled
- If TCP/IP stack is full, `send` is **blocking**
- If TCP/IP stack is almost full, `send` returns immediately, but only a **part** of the message will be handled

In [ ]:
from socket import *
serverName = '127.0.0.1'
serverPort = 1235
clientSocket = socket(AF_INET,SOCK_STREAM)
clientSocket.connect((serverName,serverPort))
message = input('lowercase sentence:')
b_message = message.encode('utf-8')
bytes_sent = 0
while bytes_sent < len(b_message):
    message_remaining = b_message[bytes_sent:]
    bytes_sent += clientSocket.send(message_remaining)
modifiedMessage = clientSocket.recv(4096)
print(modifiedMessage.decode('utf-8'))
clientSocket.close()

In [ ]:
from socket import *
serverName = '127.0.0.1'
serverPort = 1240
clientSocket = socket(AF_INET,SOCK_STREAM)
clientSocket.connect((serverName,serverPort))
message = input('lowercase sentence:')
b_message = message.encode('utf-8')
clientSocket.sendall(b_message)
modifiedMessage = clientSocket.recv(4096)
print(modifiedMessage.decode('utf-8'))
clientSocket.close()

# TCP Recv issues
- If TCP/IP stack is full enough, `recv` returns immediately, and the **specified size** message will be delivered
- If TCP/IP stack is empty, `recv` is **blocking**
- If TCP/IP stack is not empty, `recv` returns immediately, but only a **fraction** of the specified size will be delivered

---

# TCP `recvall`?

## Deciding when all data is received is application specific

- Fixed size messages
- Message size is announced before data is sent
- Special delimiters announce end of data

In [ ]:
from struct import *
from socket import *

header_struct = Struct('!I')

def recvall(sock, length):
    blocks = []
    while length:
        block = sock.recv(length)
        if not block:
            raise EOFError('socket closed with %d bytes left'
                           ' in this block'.format(length))
        length -= len(block)
        blocks.append(block)
    return b''.join(blocks)

def put_block(sock, message):
    block_length = len(message)
    sock.sendall(header_struct.pack(block_length))
    sock.sendall(message)
    
def get_block(sock):
    data = recvall(sock, header_struct.size)
    (block_length,) = header_struct.unpack(data)
    return recvall(sock, block_length)

serverName = '127.0.0.1'
serverPort = 1241
clientSocket = socket(AF_INET,SOCK_STREAM)
clientSocket.connect((serverName,serverPort))
message = input('lowercase sentence:')
b_message = message.encode('utf-8')
put_block(clientSocket,b_message)
modifiedMessage = get_block(clientSocket)
print(modifiedMessage.decode('utf-8'))
clientSocket.close()

# TCP Client (with Function)

In [ ]:
from socket import *
serverName = '127.0.0.1'
serverPort = 1234

def make_request(message):
    clientSocket = socket(AF_INET,SOCK_STREAM)
    clientSocket.connect((serverName,serverPort))
    clientSocket.sendall(message.encode('utf-8'))
    modifiedMessage = clientSocket.recv(4096)
    print(modifiedMessage.decode('utf-8'))
    clientSocket.close()
    
message = input('lowercase sentence:')
make_request(message)

# Multi-request Sequential TCP client

In [ ]:
from socket import *
serverName = '127.0.0.1'
serverPort = 1234

def make_request(message):
    clientSocket = socket(AF_INET,SOCK_STREAM)
    clientSocket.connect((serverName,serverPort))
    clientSocket.sendall(message.encode('utf-8'))
    modifiedMessage = clientSocket.recv(4096)
    print(modifiedMessage.decode('utf-8'))
    clientSocket.close()
    
message = input('lowercase sentence:')
for index in range(50):
    make_request(message)

# MultiThread TCP Client

In [ ]:
from socket import *
from threading import *

serverName = '127.0.0.1'
serverPort = 1234

def make_request(message):
    clientSocket = socket(AF_INET,SOCK_STREAM)
    clientSocket.connect((serverName,serverPort))
    clientSocket.send(message.encode('utf-8'))
    modifiedMessage = clientSocket.recv(4096)
    print(modifiedMessage.decode('utf-8'))
    clientSocket.close()
    
message = input('lowercase sentence:')
for i in range(50):
    Thread(target=make_request, args=(message,)).start()

# Multi-thread TCP client (with minimal error management)

In [ ]:
from socket import *
from threading import *
import logging

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

serverName = '127.0.0.1'
serverPort = 1241

def make_request(message):
    clientSocket = socket(AF_INET,SOCK_STREAM)
    try:
        clientSocket.connect((serverName,serverPort))
    except error:
        logging.info("Connect failed, aborting")
        clientSocket.close()
        return
    try:
        clientSocket.send(message.encode('utf-8'))
    except error:
        logging.info("Send failed, aborting")
        clientSocket.close()
        return
    try:
        modifiedMessage = clientSocket.recv(4096)
    except error:
        logging.info("Recv failed, aborting")
        clientSocket.close()
        return
    logging.info(modifiedMessage.decode('utf-8'))
    clientSocket.close()
    
message = input('lowercase sentence:')
for i in range(50):
    Thread(target=make_request, args=(message,)).start()